# SQL로 분석하는 마케팅 A/B 테스트

> **목적**: 동일한 588K 유저 데이터셋을 **SQL(DuckDB)**로 분석합니다.
> Python 노트북([case-study-marketing-ab-test.ipynb](./case-study-marketing-ab-test.ipynb))의 핵심 분석을 SQL로 재구현하여,
> 도구에 종속되지 않는 분석 역량을 보여줍니다.
>
> **사용 기술**: DuckDB (in-process OLAP), Window Functions, CTEs, CASE WHEN

---

## 이 노트북에서 보여주는 SQL 패턴

| # | 패턴 | 실무 용도 |
|---|------|----------|
| 1 | `GROUP BY` + 집계 | 기본 KPI 산출 |
| 2 | `CASE WHEN` + 버케팅 | 세그먼트 생성 |
| 3 | `WITH` (CTE) | 복잡한 쿼리 구조화 |
| 4 | Window Functions | 분위수, 순위, 누적 계산 |
| 5 | Self-JOIN | A/B 그룹 비교 (피벗) |
| 6 | 서브쿼리 + `HAVING` | 조건부 필터링 |
| 7 | 비즈니스 임팩트 쿼리 | 매출 영향 추정 |

In [1]:
import duckdb

con = duckdb.connect()

# CSV를 테이블로 로드 (컬럼명 정규화 + converted를 INTEGER로 캐스팅)
con.execute("""
    CREATE TABLE ab_test AS
    SELECT 
        "user id"                    AS user_id,
        "test group"                 AS test_group,
        CAST("converted" AS INTEGER) AS converted,
        "total ads"                  AS total_ads,
        "most ads day"               AS most_ads_day,
        "most ads hour"              AS most_ads_hour
    FROM read_csv('marketing_AB.csv', header=true, auto_detect=true)
""")

con.sql('SELECT COUNT(*) AS total_rows FROM ab_test')

┌────────────┐
│ total_rows │
│   int64    │
├────────────┤
│     588101 │
└────────────┘

---

## 1. 기본 집계 — 그룹별 KPI (`GROUP BY`)

In [2]:
con.sql("""
    SELECT 
        test_group,
        COUNT(*)                              AS users,
        SUM(converted)                        AS conversions,
        ROUND(AVG(converted) * 100, 3)        AS cvr_pct,
        ROUND(MEDIAN(total_ads), 1)           AS median_ads,
        ROUND(AVG(total_ads), 1)              AS avg_ads,
        MAX(total_ads)                        AS max_ads
    FROM ab_test
    GROUP BY test_group
    ORDER BY users DESC
""")

┌────────────┬────────┬─────────────┬─────────┬────────────┬─────────┬─────────┐
│ test_group │ users  │ conversions │ cvr_pct │ median_ads │ avg_ads │ max_ads │
│  varchar   │ int64  │   int128    │ double  │   double   │ double  │  int64  │
├────────────┼────────┼─────────────┼─────────┼────────────┼─────────┼─────────┤
│ ad         │ 564577 │       14423 │   2.555 │       13.0 │    24.8 │    2065 │
│ psa        │  23524 │         420 │   1.785 │       12.0 │    24.8 │     907 │
└────────────┴────────┴─────────────┴─────────┴────────────┴─────────┴─────────┘

---

## 2. SRM 검증 — 기대 분배와 관측값 비교 (`WITH` + 수식)

In [3]:
con.sql("""
    WITH observed AS (
        SELECT 
            test_group,
            COUNT(*) AS obs_n
        FROM ab_test
        GROUP BY test_group
    ),
    total AS (
        SELECT SUM(obs_n) AS total_n FROM observed
    ),
    expected_splits(test_group, expected_ratio) AS (
        VALUES ('ad', 0.96), ('psa', 0.04)
    ),
    srm_calc AS (
        SELECT 
            o.test_group,
            o.obs_n                                       AS observed,
            ROUND(t.total_n * e.expected_ratio)           AS expected,
            ROUND(POWER(o.obs_n - t.total_n * e.expected_ratio, 2) 
                  / (t.total_n * e.expected_ratio), 4)    AS chi2_component
        FROM observed o
        JOIN total t ON 1=1
        JOIN expected_splits e ON o.test_group = e.test_group
    )
    SELECT 
        *,
        SUM(chi2_component) OVER () AS chi2_total,
        CASE 
            WHEN SUM(chi2_component) OVER () < 3.841 THEN 'Healthy (p > 0.05)'
            WHEN SUM(chi2_component) OVER () < 10.828 THEN 'Warning (p < 0.001)'
            ELSE 'Blocked (p < 0.00001)'
        END AS srm_status
    FROM srm_calc
""")

┌────────────┬──────────┬───────────────┬────────────────┬────────────┬────────────────────┐
│ test_group │ observed │   expected    │ chi2_component │ chi2_total │     srm_status     │
│  varchar   │  int64   │ decimal(38,0) │     double     │   double   │      varchar       │
├────────────┼──────────┼───────────────┼────────────────┼────────────┼────────────────────┤
│ ad         │   564577 │        564577 │            0.0 │        0.0 │ Healthy (p > 0.05) │
│ psa        │    23524 │         23524 │            0.0 │        0.0 │ Healthy (p > 0.05) │
└────────────┴──────────┴───────────────┴────────────────┴────────────┴────────────────────┘

chi2_total이 매우 작고 (< 3.841) **Healthy** 상태입니다. 96/4 기대 split과 관측값이 일치합니다.

---

## 3. 세그먼트 생성 — 노출량 버케팅 (`CASE WHEN`)

In [4]:
con.sql("""
    SELECT 
        CASE 
            WHEN total_ads BETWEEN 1 AND 5    THEN '01. 1-5'
            WHEN total_ads BETWEEN 6 AND 10   THEN '02. 6-10'
            WHEN total_ads BETWEEN 11 AND 20  THEN '03. 11-20'
            WHEN total_ads BETWEEN 21 AND 50  THEN '04. 21-50'
            WHEN total_ads BETWEEN 51 AND 100 THEN '05. 51-100'
            WHEN total_ads BETWEEN 101 AND 200 THEN '06. 101-200'
            WHEN total_ads > 200              THEN '07. 200+'
        END AS exposure_bucket,
        test_group,
        COUNT(*)                          AS users,
        SUM(converted)                    AS conversions,
        ROUND(AVG(converted) * 100, 2)    AS cvr_pct
    FROM ab_test
    GROUP BY exposure_bucket, test_group
    ORDER BY exposure_bucket, test_group
""")

┌─────────────────┬────────────┬────────┬─────────────┬─────────┐
│ exposure_bucket │ test_group │ users  │ conversions │ cvr_pct │
│     varchar     │  varchar   │ int64  │   int128    │ double  │
├─────────────────┼────────────┼────────┼─────────────┼─────────┤
│ 01. 1-5         │ ad         │ 169962 │         427 │    0.25 │
│ 01. 1-5         │ psa        │   7861 │          22 │    0.28 │
│ 02. 6-10        │ ad         │  79537 │         386 │    0.49 │
│ 02. 6-10        │ psa        │   3415 │          23 │    0.67 │
│ 03. 11-20       │ ad         │ 123334 │        1036 │    0.84 │
│ 03. 11-20       │ psa        │   4150 │          34 │    0.82 │
│ 04. 21-50       │ ad         │ 125541 │        3660 │    2.92 │
│ 04. 21-50       │ psa        │   5235 │         114 │    2.18 │
│ 05. 51-100      │ ad         │  44149 │        5135 │   11.63 │
│ 05. 51-100      │ psa        │   1853 │         107 │    5.77 │
│ 06. 101-200     │ ad         │  16360 │        2892 │   17.68 │
│ 06. 101-

---

## 4. A/B 비교 피벗 — Self-JOIN으로 Lift 계산

In [5]:
con.sql("""
    WITH bucketed AS (
        SELECT 
            CASE 
                WHEN total_ads BETWEEN 1 AND 5    THEN '01. 1-5'
                WHEN total_ads BETWEEN 6 AND 10   THEN '02. 6-10'
                WHEN total_ads BETWEEN 11 AND 20  THEN '03. 11-20'
                WHEN total_ads BETWEEN 21 AND 50  THEN '04. 21-50'
                WHEN total_ads BETWEEN 51 AND 100 THEN '05. 51-100'
                WHEN total_ads BETWEEN 101 AND 200 THEN '06. 101-200'
                WHEN total_ads > 200              THEN '07. 200+'
            END AS bucket,
            test_group,
            COUNT(*)           AS users,
            SUM(converted)     AS conversions,
            AVG(converted)     AS cvr
        FROM ab_test
        GROUP BY bucket, test_group
    )
    SELECT 
        a.bucket                                              AS exposure,
        a.users                                               AS ad_users,
        ROUND(a.cvr * 100, 2)                                 AS ad_cvr_pct,
        p.users                                               AS psa_users,
        ROUND(p.cvr * 100, 2)                                 AS psa_cvr_pct,
        ROUND((a.cvr - p.cvr) * 100, 3)                      AS lift_pp,
        ROUND((a.cvr / NULLIF(p.cvr, 0) - 1) * 100, 1)      AS relative_lift_pct,
        -- Z-test approximation
        ROUND(
            (a.cvr - p.cvr) 
            / SQRT(
                ((a.conversions + p.conversions)::DOUBLE / (a.users + p.users))
                * (1 - (a.conversions + p.conversions)::DOUBLE / (a.users + p.users))
                * (1.0 / a.users + 1.0 / p.users)
              ), 2)                                           AS z_score
    FROM bucketed a
    JOIN bucketed p ON a.bucket = p.bucket
    WHERE a.test_group = 'ad' AND p.test_group = 'psa'
    ORDER BY a.bucket
""")

┌─────────────┬──────────┬────────────┬───────────┬─────────────┬─────────┬───────────────────┬─────────┐
│  exposure   │ ad_users │ ad_cvr_pct │ psa_users │ psa_cvr_pct │ lift_pp │ relative_lift_pct │ z_score │
│   varchar   │  int64   │   double   │   int64   │   double    │ double  │      double       │ double  │
├─────────────┼──────────┼────────────┼───────────┼─────────────┼─────────┼───────────────────┼─────────┤
│ 01. 1-5     │   169962 │       0.25 │      7861 │        0.28 │  -0.029 │             -10.2 │   -0.49 │
│ 02. 6-10    │    79537 │       0.49 │      3415 │        0.67 │  -0.188 │             -27.9 │   -1.54 │
│ 03. 11-20   │   123334 │       0.84 │      4150 │        0.82 │   0.021 │               2.5 │    0.14 │
│ 04. 21-50   │   125541 │       2.92 │      5235 │        2.18 │   0.738 │              33.9 │    3.12 │
│ 05. 51-100  │    44149 │      11.63 │      1853 │        5.77 │   5.857 │             101.4 │    7.77 │
│ 06. 101-200 │    16360 │      17.68 │       

**Self-JOIN**으로 ad/psa를 같은 행에 배치하고 Lift와 Z-score를 한 쿼리로 계산합니다.

|z| > 1.96이면 95% 유의 → 51-100, 101-200 구간에서 효과가 가장 큽니다.

---

## 5. Window Functions — 분위수별 전환율 + 누적 분포

In [6]:
con.sql("""
    WITH user_deciles AS (
        SELECT 
            *,
            NTILE(10) OVER (PARTITION BY test_group ORDER BY total_ads) AS exposure_decile
        FROM ab_test
    )
    SELECT 
        test_group,
        exposure_decile,
        COUNT(*)                                          AS users,
        MIN(total_ads)                                    AS min_ads,
        MAX(total_ads)                                    AS max_ads,
        ROUND(AVG(converted) * 100, 2)                    AS cvr_pct,
        -- Cumulative conversion rate up to this decile
        ROUND(
            SUM(SUM(converted)) OVER (PARTITION BY test_group ORDER BY exposure_decile)
            * 100.0
            / SUM(COUNT(*)) OVER (PARTITION BY test_group ORDER BY exposure_decile)
        , 2)                                              AS cumul_cvr_pct
    FROM user_deciles
    GROUP BY test_group, exposure_decile
    ORDER BY test_group, exposure_decile
""")

┌────────────┬─────────────────┬───────┬─────────┬─────────┬─────────┬───────────────┐
│ test_group │ exposure_decile │ users │ min_ads │ max_ads │ cvr_pct │ cumul_cvr_pct │
│  varchar   │      int64      │ int64 │  int64  │  int64  │ double  │    double     │
├────────────┼─────────────────┼───────┼─────────┼─────────┼─────────┼───────────────┤
│ ad         │               1 │ 56458 │       1 │       2 │     0.2 │           0.2 │
│ ad         │               2 │ 56458 │       2 │       3 │    0.19 │          0.19 │
│ ad         │               3 │ 56458 │       3 │       5 │    0.36 │          0.25 │
│ ad         │               4 │ 56458 │       5 │       8 │    0.43 │          0.29 │
│ ad         │               5 │ 56458 │       8 │      13 │    0.67 │          0.37 │
│ ad         │               6 │ 56458 │      13 │      17 │    0.77 │          0.43 │
│ ad         │               7 │ 56458 │      17 │      24 │    1.21 │          0.55 │
│ ad         │               8 │ 56457 │   

`NTILE(10)`로 10분위를 만들고, `SUM() OVER (ORDER BY ...)` 로 **누적 전환율**을 계산합니다.

이 패턴은 실무에서 **Retention curve**, **Revenue concentration (Pareto)** 분석에 자주 사용됩니다.

---

## 6. 요일 × 시간대 교차 분석 — 피벗 테이블 (`CASE WHEN` + `GROUP BY`)

In [7]:
con.sql("""
    WITH hour_bucket AS (
        SELECT *,
            CASE 
                WHEN most_ads_hour BETWEEN 0 AND 6   THEN 'Night'
                WHEN most_ads_hour BETWEEN 7 AND 12  THEN 'Morning'
                WHEN most_ads_hour BETWEEN 13 AND 18 THEN 'Afternoon'
                ELSE                                      'Evening'
            END AS time_slot
        FROM ab_test
        WHERE test_group = 'ad'
    )
    SELECT 
        most_ads_day                                          AS day,
        ROUND(AVG(CASE WHEN time_slot='Morning'   THEN converted END) * 100, 2) AS morning_cvr,
        ROUND(AVG(CASE WHEN time_slot='Afternoon' THEN converted END) * 100, 2) AS afternoon_cvr,
        ROUND(AVG(CASE WHEN time_slot='Evening'   THEN converted END) * 100, 2) AS evening_cvr,
        ROUND(AVG(CASE WHEN time_slot='Night'     THEN converted END) * 100, 2) AS night_cvr,
        COUNT(*)                                              AS total_users
    FROM hour_bucket
    GROUP BY most_ads_day
    ORDER BY 
        CASE most_ads_day
            WHEN 'Monday'    THEN 1
            WHEN 'Tuesday'   THEN 2
            WHEN 'Wednesday' THEN 3
            WHEN 'Thursday'  THEN 4
            WHEN 'Friday'    THEN 5
            WHEN 'Saturday'  THEN 6
            WHEN 'Sunday'    THEN 7
        END
""")

┌───────────┬─────────────┬───────────────┬─────────────┬───────────┬─────────────┐
│    day    │ morning_cvr │ afternoon_cvr │ evening_cvr │ night_cvr │ total_users │
│  varchar  │   double    │    double     │   double    │  double   │    int64    │
├───────────┼─────────────┼───────────────┼─────────────┼───────────┼─────────────┤
│ Monday    │         3.1 │          3.76 │        3.08 │      1.63 │       83571 │
│ Tuesday   │        2.53 │          3.74 │        2.83 │       1.5 │       74572 │
│ Wednesday │        2.27 │          2.98 │        2.43 │      0.88 │       77418 │
│ Thursday  │        1.84 │          2.34 │        2.32 │      1.81 │       79077 │
│ Friday    │        1.96 │          2.46 │        2.39 │      1.27 │       88805 │
│ Saturday  │        1.73 │          2.15 │        2.65 │      1.56 │       78802 │
│ Sunday    │        1.81 │          2.47 │        3.58 │       1.4 │       82332 │
└───────────┴─────────────┴───────────────┴─────────────┴───────────┴───────

`CASE WHEN` 안에 집계를 넣어 **SQL 피벗 테이블**을 만듭니다. Tableau/Looker에서 "행: 요일, 열: 시간대, 값: 전환율"로 보는 것과 같은 결과입니다.

---

## 7. 고가치 유저 탐색 — 서브쿼리 + `HAVING`

In [8]:
con.sql("""
    -- 어떤 세그먼트에서 ad 그룹의 전환율이 가장 높은가?
    -- 조건: 최소 1,000명 이상인 세그먼트만 (노이즈 제거)
    WITH segment_stats AS (
        SELECT 
            most_ads_day                          AS day,
            CASE 
                WHEN total_ads <= 20  THEN 'low'
                WHEN total_ads <= 100 THEN 'mid'
                ELSE                      'high'
            END                                   AS exposure_level,
            COUNT(*)                              AS users,
            SUM(converted)                        AS conversions,
            ROUND(AVG(converted) * 100, 2)        AS cvr_pct
        FROM ab_test
        WHERE test_group = 'ad'
        GROUP BY day, exposure_level
        HAVING COUNT(*) >= 1000
    )
    SELECT *
    FROM segment_stats
    WHERE cvr_pct = (SELECT MAX(cvr_pct) FROM segment_stats)
       OR cvr_pct = (SELECT MIN(cvr_pct) FROM segment_stats)
    ORDER BY cvr_pct DESC
""")

┌──────────┬────────────────┬───────┬─────────────┬─────────┐
│   day    │ exposure_level │ users │ conversions │ cvr_pct │
│ varchar  │    varchar     │ int64 │   int128    │ double  │
├──────────┼────────────────┼───────┼─────────────┼─────────┤
│ Tuesday  │ high           │  2618 │         560 │   21.39 │
│ Thursday │ low            │ 53875 │         211 │    0.39 │
│ Saturday │ low            │ 50490 │         196 │    0.39 │
└──────────┴────────────────┴───────┴─────────────┴─────────┘

**`HAVING`**으로 소규모 세그먼트를 제거하고, 서브쿼리로 **최고/최저 전환율 세그먼트**를 식별합니다.

이 패턴은 실무에서 "어디에 집중할까?" 의사결정에 직접 사용됩니다.

---

## 8. 비즈니스 임팩트 추정 — 매출 영향 시뮬레이션

In [9]:
con.sql("""
    -- 가정: 전환당 매출 $50 (e-commerce 평균 객단가)
    WITH impact AS (
        SELECT 
            CASE 
                WHEN total_ads BETWEEN 1 AND 5    THEN '01. 1-5'
                WHEN total_ads BETWEEN 6 AND 10   THEN '02. 6-10'
                WHEN total_ads BETWEEN 11 AND 20  THEN '03. 11-20'
                WHEN total_ads BETWEEN 21 AND 50  THEN '04. 21-50'
                WHEN total_ads BETWEEN 51 AND 100 THEN '05. 51-100'
                WHEN total_ads BETWEEN 101 AND 200 THEN '06. 101-200'
                WHEN total_ads > 200              THEN '07. 200+'
            END AS bucket,
            test_group,
            COUNT(*)           AS users,
            AVG(converted)     AS cvr
        FROM ab_test
        GROUP BY bucket, test_group
    ),
    lift_by_bucket AS (
        SELECT 
            a.bucket,
            a.users                          AS ad_users,
            a.cvr                            AS ad_cvr,
            p.cvr                            AS psa_cvr,
            (a.cvr - p.cvr)                  AS lift,
            -- Incremental conversions from ad vs PSA
            ROUND(a.users * (a.cvr - p.cvr)) AS incremental_conversions
        FROM impact a
        JOIN impact p ON a.bucket = p.bucket
        WHERE a.test_group = 'ad' AND p.test_group = 'psa'
    )
    SELECT 
        bucket                                                    AS exposure,
        ad_users,
        ROUND(lift * 100, 3)                                      AS lift_pp,
        incremental_conversions,
        incremental_conversions * 50                               AS incremental_revenue_usd,
        -- Cost efficiency: how many users needed per 1 incremental conversion
        CASE WHEN incremental_conversions > 0 
             THEN ROUND(ad_users::DOUBLE / incremental_conversions, 0)
             ELSE NULL 
        END                                                       AS users_per_conversion,
        -- Revenue contribution %
        ROUND(
            incremental_conversions * 100.0 
            / NULLIF(SUM(CASE WHEN incremental_conversions > 0 THEN incremental_conversions ELSE 0 END) OVER (), 0)
        , 1)                                                      AS revenue_contribution_pct
    FROM lift_by_bucket
    ORDER BY bucket
""")

┌─────────────┬──────────┬─────────┬─────────────────────────┬─────────────────────────┬──────────────────────┬──────────────────────────┐
│  exposure   │ ad_users │ lift_pp │ incremental_conversions │ incremental_revenue_usd │ users_per_conversion │ revenue_contribution_pct │
│   varchar   │  int64   │ double  │         double          │         double          │        double        │          double          │
├─────────────┼──────────┼─────────┼─────────────────────────┼─────────────────────────┼──────────────────────┼──────────────────────────┤
│ 01. 1-5     │   169962 │  -0.029 │                   -49.0 │                 -2450.0 │                 NULL │                     -1.0 │
│ 02. 6-10    │    79537 │  -0.188 │                  -150.0 │                 -7500.0 │                 NULL │                     -3.2 │
│ 03. 11-20   │   123334 │   0.021 │                    26.0 │                  1300.0 │               4744.0 │                      0.6 │
│ 04. 21-50   │   125541 │ 

In [10]:
# === 전체 요약 ===
con.sql("""
    WITH overall AS (
        SELECT 
            test_group,
            COUNT(*)       AS users,
            AVG(converted) AS cvr
        FROM ab_test
        GROUP BY test_group
    )
    SELECT 
        a.users                                        AS ad_users,
        ROUND(a.cvr * 100, 3)                          AS ad_cvr_pct,
        ROUND(p.cvr * 100, 3)                          AS psa_cvr_pct,
        ROUND((a.cvr - p.cvr) * 100, 3)                AS lift_pp,
        ROUND((a.cvr / p.cvr - 1) * 100, 1)            AS relative_lift_pct,
        ROUND(a.users * (a.cvr - p.cvr))               AS total_incremental_conversions,
        ROUND(a.users * (a.cvr - p.cvr)) * 50          AS total_incremental_revenue_usd,
        -- If we could move low-exposure users (1-20 ads, ~60%) to mid-exposure (21-50)
        -- Expected additional lift: from ~0%p to ~0.74%p
        ROUND(a.users * 0.6 * 0.0074)                  AS potential_additional_conversions,
        ROUND(a.users * 0.6 * 0.0074) * 50             AS potential_additional_revenue_usd
    FROM overall a, overall p
    WHERE a.test_group = 'ad' AND p.test_group = 'psa'
""")

┌──────────┬────────────┬─────────────┬─────────┬───────────────────┬───────────────────────────────┬───────────────────────────────┬──────────────────────────────────┬──────────────────────────────────┐
│ ad_users │ ad_cvr_pct │ psa_cvr_pct │ lift_pp │ relative_lift_pct │ total_incremental_conversions │ total_incremental_revenue_usd │ potential_additional_conversions │ potential_additional_revenue_usd │
│  int64   │   double   │   double    │ double  │      double       │            double             │            double             │          decimal(26,0)           │          decimal(36,0)           │
├──────────┼────────────┼─────────────┼─────────┼───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────────────────────────────┼──────────────────────────────────┤
│   564577 │      2.555 │       1.785 │   0.769 │              43.1 │                        4343.0 │                      217150.0 │                             2507 │                

---

## 9. Funnel 패턴 — 노출 → 전환 단계별 이탈률

In [11]:
con.sql("""
    -- Funnel: 전체 유저 → 일정 노출 이상 → 전환
    -- 실무에서 "가입 → 활성화 → 구매" 퍼널과 동일한 패턴
    WITH funnel AS (
        SELECT 
            COUNT(*)                                               AS total_users,
            SUM(CASE WHEN total_ads >= 10 THEN 1 ELSE 0 END)      AS reached_10_ads,
            SUM(CASE WHEN total_ads >= 50 THEN 1 ELSE 0 END)      AS reached_50_ads,
            SUM(CASE WHEN total_ads >= 100 THEN 1 ELSE 0 END)     AS reached_100_ads,
            SUM(converted)                                         AS converted_users
        FROM ab_test
        WHERE test_group = 'ad'
    )
    SELECT 
        'All Ad Users'     AS stage, total_users     AS users, 100.0 AS pct FROM funnel
    UNION ALL SELECT 
        '10+ Ads Seen',    reached_10_ads,  ROUND(reached_10_ads  * 100.0 / total_users, 1) FROM funnel
    UNION ALL SELECT 
        '50+ Ads Seen',    reached_50_ads,  ROUND(reached_50_ads  * 100.0 / total_users, 1) FROM funnel
    UNION ALL SELECT 
        '100+ Ads Seen',   reached_100_ads, ROUND(reached_100_ads * 100.0 / total_users, 1) FROM funnel
    UNION ALL SELECT 
        'Converted',       converted_users, ROUND(converted_users * 100.0 / total_users, 1) FROM funnel
""")

┌───────────────┬────────┬────────┐
│     stage     │ users  │  pct   │
│    varchar    │ int128 │ double │
├───────────────┼────────┼────────┤
│ All Ad Users  │ 564577 │  100.0 │
│ 10+ Ads Seen  │ 326511 │   57.8 │
│ 50+ Ads Seen  │  68148 │   12.1 │
│ 100+ Ads Seen │  22439 │    4.0 │
│ Converted     │  14423 │    2.6 │
└───────────────┴────────┴────────┘

이 **Funnel 쿼리 패턴**은 실무에서 가장 자주 사용되는 SQL 패턴 중 하나입니다:
- 가입 → 온보딩 → 첫 구매 → 재구매
- 페이지 방문 → 장바구니 → 결제 시도 → 결제 완료

---

## 10. 최종 의사결정 쿼리 — PM에게 보내는 한 장 요약

In [12]:
con.sql("""
    WITH metrics AS (
        SELECT 
            test_group,
            COUNT(*)       AS n,
            AVG(converted) AS cvr
        FROM ab_test
        GROUP BY test_group
    ),
    comparison AS (
        SELECT 
            a.cvr - p.cvr                     AS abs_lift,
            (a.cvr / p.cvr - 1)               AS rel_lift,
            (a.cvr - p.cvr) 
            / SQRT(
                ((a.n * a.cvr + p.n * p.cvr) / (a.n + p.n))
                * (1 - (a.n * a.cvr + p.n * p.cvr) / (a.n + p.n))
                * (1.0/a.n + 1.0/p.n)
              )                               AS z_score,
            a.n                               AS ad_users,
            a.cvr                             AS ad_cvr,
            p.cvr                             AS psa_cvr
        FROM metrics a, metrics p
        WHERE a.test_group = 'ad' AND p.test_group = 'psa'
    )
    SELECT
        '588K Marketing A/B Test'                        AS experiment,
        ROUND(psa_cvr * 100, 3) || '% → ' || ROUND(ad_cvr * 100, 3) || '%' AS cvr_change,
        '+' || ROUND(abs_lift * 100, 3) || '%p'          AS absolute_lift,
        '+' || ROUND(rel_lift * 100, 1) || '%'           AS relative_lift,
        ROUND(z_score, 2)                                AS z_score,
        CASE WHEN ABS(z_score) > 2.576 THEN 'p < 0.01 ***'
             WHEN ABS(z_score) > 1.96  THEN 'p < 0.05 *'
             ELSE 'Not significant'
        END                                              AS significance,
        CASE WHEN ABS(z_score) > 1.96 AND abs_lift > 0 
             THEN 'LAUNCH'
             ELSE 'HOLD'
        END                                              AS decision,
        '$' || FORMAT('{:,.0f}', ROUND(ad_users * abs_lift) * 50) AS est_incremental_revenue
    FROM comparison
""")

┌─────────────────────────┬─────────────────┬───────────────┬───────────────┬─────────┬──────────────┬──────────┬─────────────────────────┐
│       experiment        │   cvr_change    │ absolute_lift │ relative_lift │ z_score │ significance │ decision │ est_incremental_revenue │
│         varchar         │     varchar     │    varchar    │    varchar    │ double  │   varchar    │ varchar  │         varchar         │
├─────────────────────────┼─────────────────┼───────────────┼───────────────┼─────────┼──────────────┼──────────┼─────────────────────────┤
│ 588K Marketing A/B Test │ 1.785% → 2.555% │ +0.769%p      │ +43.1%        │    7.37 │ p < 0.01 *** │ LAUNCH   │ $217,150                │
└─────────────────────────┴─────────────────┴───────────────┴───────────────┴─────────┴──────────────┴──────────┴─────────────────────────┘

---

## 요약: 이 노트북에서 사용한 SQL 패턴

| # | SQL 패턴 | 셀 | 실무 용도 |
|---|---------|:---:|----------|
| 1 | `GROUP BY` + 집계 | 1, 3 | 기본 KPI, 대시보드 메트릭 |
| 2 | `WITH` (CTE) 체이닝 | 5, 9, 17, 22 | 복잡한 분석을 읽기 쉽게 분해 |
| 3 | `CASE WHEN` 버케팅 | 7, 13 | 세그먼트 생성, 피벗 테이블 |
| 4 | Self-JOIN (A/B 비교) | 9, 17 | 실험 그룹 간 Lift 계산 |
| 5 | Window: `NTILE`, 누적 `SUM` | 11 | 분위수 분석, Pareto |
| 6 | Window: `SUM() OVER ()` | 5, 17 | 전체 대비 비율, 기여도 |
| 7 | `HAVING` + 서브쿼리 | 15 | 조건부 필터, 최고/최저 탐색 |
| 8 | `UNION ALL` 퍼널 | 20 | 단계별 이탈률 분석 |
| 9 | 수식 내장 (z-test, chi2) | 5, 9, 22 | 통계 검정을 SQL로 수행 |

### Reference

- Dataset: FavioVázquez (2020). *Marketing A/B Testing*. Kaggle. CC0 Public Domain.
- DuckDB: [duckdb.org](https://duckdb.org) — In-process OLAP database.